# Calibration

## Libraries and FPGA image

We will start by importing the *rp* (Red Pitaya) and *rp_overlay* libraries and loading the *v0.94* FPGA image.

In [ ]:
from rp_overlay import overlay
import rp
import rp_hw_calib

fpga = overlay()

Initialize the Red Pitaya calibration interface.

In [ ]:
rp_hw_calib.rp_CalibInit()

## Macros

- **Errors:**
  - RP_HW_CALIB_OK    - Success  
  - RP_HW_CALIB_ERE   - Error read from eeprom  
  - RP_HW_CALIB_EWE   - Error write to eeprom  
  - RP_HW_CALIB_ENI   - Error Calibration values are not initialized  
  - RP_HW_CALIB_EDM   - Board Model Detection Error  
  - RP_HW_CALIB_ECH   - Invalid channel parameter  
  - RP_HW_CALIB_EIP   - Invalid parameter  
  - RP_HW_CALIB_EA    - Adjust error
- **Board ID:**
  - RP_HW_PACK_ID_V1 - STEMLAB 125-14  
  - RP_HW_PACK_ID_V2 - SIGNALlab 250-12  
  - RP_HW_PACK_ID_V3 - STEMlab 125-14 4-Input  
  - RP_HW_PACK_ID_V4 - SDRlab 122-16
  - RP_HW_PACK_ID_V5 - Universal calibration
- **Fast analog inputs** - RP_CH_1_CALIB, RP_CH_2_CALIB, RP_CH_3_CALIB, RP_CH_4_CALIB

SIGNALlab 250-12 only:
- **Calibration generator gain** - RP_GAIN_CALIB_1X, RP_GAIN_CALIB_5X
- **Coupling mode** - RP_DC_CALIB, RP_AC_CALIB

## Structures

For more information regarding the specific functions and structures, please check the following two GitHub files:

- [rp_hw-calib.h](https://github.com/RedPitaya/RedPitaya/blob/master/rp-api/api-hw-calib/include/rp_hw-calib.h)
- [calib_universal.h](https://github.com/RedPitaya/RedPitaya/blob/master/rp-api/api-hw-calib/src/calib_universal.h)

In [ ]:
typedef struct{
    uint32_t gain;
    uint32_t base;
    uint8_t  precision;
    int32_t  offset;
} uint_gain_calib_t;

typedef struct{
    float       baseScale;
    uint32_t    calibValue;
    int32_t     offset;
    double      gainCalc;
} channel_calib_t;

typedef struct
{
    uint32_t aa;
    uint32_t bb;
    uint32_t pp;
    uint32_t kk;
} channel_filter_t;


typedef struct {
    char dataStructureId;
    char wpCheck;

    uint8_t fast_adc_count_1_1; // For 250-12 is DC mode
    channel_calib_t fast_adc_1_1[4];
    channel_filter_t fast_adc_filter_1_1[4];
    uint8_t fast_adc_count_1_20;
    channel_calib_t fast_adc_1_20[4]; // For 250-12 is DC mode
    channel_filter_t fast_adc_filter_1_20[4];

    uint8_t fast_adc_count_1_1_ac; // For 250-12
    channel_calib_t fast_adc_1_1_ac[4];
    uint8_t fast_adc_count_1_20_ac; // For 250-12
    channel_calib_t fast_adc_1_20_ac[4];

    uint8_t fast_dac_count_x1;
    channel_calib_t fast_dac_x1[2];

    uint8_t fast_dac_count_x5; // For 250-12
    channel_calib_t fast_dac_x5[2];
} rp_calib_params_t;

## Read calibration values
Read calibration ID numbers in the order they appear in EEPROM.

In [ ]:
for i in range(1, 65, 1):
    print(f"{i}  {rp_hw_calib.rp_GetNameOfUniversalId(i)[1]}")

Here is how we can read Calibration data directly from EEPROM. Please note that one ID number can stretch across multiple EEPROM bytes.

In [ ]:
raw_data = rp_hw_calib.rp_CalibGetEEPROM(False)
print(raw_data)

data_arr = rp_hw_calib.uint8Arr_frompointer(raw_data[1])

for n in range(raw_data[2]):
    print(hex(data_arr[n]), end=",")
print("")

**LV calibration values:**  
This function automatically recalculates gain to a floating point (from EEPROM integer values gain, base, and precision).

In [ ]:
ch1_lv_calib = rp_hw_calib.rp_CalibGetFastADCCalibValue(0,0)    # Channel, AC/DC mode
ch2_lv_calib = rp_hw_calib.rp_CalibGetFastADCCalibValue(1,0)

print(f"LV ADC[1]: Gain = {ch1_lv_calib[1]}, Offset = {ch1_lv_calib[2]}\n")
print(f"LV ADC[2]: Gain = {ch2_lv_calib[1]}, Offset = {ch2_lv_calib[2]}\n")

**HV calibration values:**  
This function automatically recalculates gain to a floating point (from EEPROM integer values gain, base, and precision).

In [ ]:
ch1_hv_calib = rp_hw_calib.rp_CalibGetFastADCCalibValue_1_20(0,0)    # Channel, AC/DC mode
ch2_hv_calib = rp_hw_calib.rp_CalibGetFastADCCalibValue_1_20(1,0)

print(f"HV ADC[1]: Gain = {ch1_hv_calib[1]}, Offset = {ch1_hv_calib[2]}\n")
print(f"HV ADC[2]: Gain = {ch2_hv_calib[1]}, Offset = {ch2_hv_calib[2]}\n")

**DAC calibration values:**  
This function automatically recalculates gain to a floating point (from EEPROM integer values gain, base, and precision).

In [ ]:
out1_calib = rp_hw_calib.rp_CalibGetFastDACCalibValue(0,0)      # Channel, Generator gain mode
out2_calib = rp_hw_calib.rp_CalibGetFastDACCalibValue(1,0)

print(f"DAC[1]: Gain = {out1_calib[1]}, Offset = {out1_calib[2]}\n")
print(f"DAC[2]: Gain = {out2_calib[1]}, Offset = {out2_calib[2]}\n")

**RAW calibration values:**  
The values are read directly from EEPROM, and no gain recalculation is performed.  
The functions to read HV ADC and DAC parameters are almost identical.

In [ ]:
t = rp_hw_calib.new_p_uint_gain_calib_t()
res = rp_hw_calib.rp_CalibGetFastADCCalibValueI(0,0,t)  # Channel, AC/DC mode, array 

print(f"LV ADC[1]: Gain = {t.gain}, Base = {t.base}, Precision = {t.precision}, Offset = {t.offset}\n")

**Reading FPGA filter values:**

In [ ]:
t = rp_hw_calib.new_p_channel_filter_t()

# Channel 1 LV FPGA filter
rp_hw_calib.rp_CalibGetFastADCFilter(0, t)    # Channel, array
print(f"LV ADC[1] filter parameters: AA = {t.aa}, BB = {t.bb}, PP = {t.pp}, KK = {t.kk}\n")

# Channel 1 HV FPGA filter
rp_hw_calib.rp_CalibGetFastADCFilter_1_20(0, t)
print(f"HV ADC[1] filter parameters: AA = {t.aa}, BB = {t.bb}, PP = {t.pp}, KK = {t.kk}\n")

## <font color='red'>DANGER ZONE</font>
### Backup
Here is how to back up the calibration settings in case you want to try out the danger zone.

In [ ]:
current_settings = rp_hw_calib.rp_GetCalibrationSettings()

Restoring calibration parameters from backup:

In [ ]:
rp_hw_calib.rp_CalibrationWriteParams(current_settings,False)

### Reseting calibration
With the following commands, we can reset the calibration values of the Red Pitaya. After the reset, please restart JupyterLab for the changes to take effect.  

If trying things out, do NOT close Jupyter Notebook, as you will lose the Backup variable.

In [ ]:
# Reset to defaul values
rp_hw_calib.rp_CalibrationReset(False,False)

# Reset to factory calibration
rp_hw_calib.rp_CalibrationFactoryReset(False)

In [ ]:
# Checking the reset values
ch1_lv_calib = rp_hw_calib.rp_CalibGetFastADCCalibValue(0,0)    # Channel, AC/DC mode
ch2_lv_calib = rp_hw_calib.rp_CalibGetFastADCCalibValue(1,0)

print(f"LV ADC[1]: Gain = {ch1_lv_calib[1]}, Offset = {ch1_lv_calib[2]}\n")
print(f"LV ADC[2]: Gain = {ch2_lv_calib[1]}, Offset = {ch2_lv_calib[2]}\n")

### Change calibration values
In this example, we will read the calibration values and then write the same parameters back to EEPROM.

In [ ]:
# Get current calibration settings
rp_calib = rp_hw_calib.rp_GetCalibrationSettings()

calib_1 = rp_hw_calib.cCalibArr_getitem(rp_calib.fast_adc_1_1,0)  # Channels 0,1
calib_2 = rp_hw_calib.cCalibArr_getitem(rp_calib.fast_adc_1_1,1)

print(calib_1.gainCalc)
print(calib_2.gainCalc)

# Save values for later
calib_1_temp_gC = calib_1.gainCalc
calib_2_temp_gC = calib_2.gainCalc

Changing two parameters:

In [ ]:
# New values
calib_1.gainCalc = 1.1
calib_2.gainCalc = 1.1

# Change the calibration values
rp_hw_calib.cCalibArr_setitem(rp_calib.fast_adc_1_1, 0, calib_1)
rp_hw_calib.cCalibArr_setitem(rp_calib.fast_adc_1_1, 1, calib_2)

# Calibration data is written to EEPROM
rp_hw_calib.rp_CalibrationWriteParams(rp_calib,False)

Setting the parameters into temporary memory and printing the calibration.

In [ ]:
rp_hw_calib.rp_CalibrationSetParams(rp_calib)

# Only works in terminal
rp_hw_calib.rp_CalibPrint(rp_calib)
print(rp_calib)

Reading new values:

In [ ]:
rp_calib = rp_hw_calib.rp_GetCalibrationSettings()

calib_1 = rp_hw_calib.cCalibArr_getitem(rp_calib.fast_adc_1_1,0)  # Channels 0,1,2,3
calib_2 = rp_hw_calib.cCalibArr_getitem(rp_calib.fast_adc_1_1,1)

print(calib_1.gainCalc)
print(calib_2.gainCalc)

Restoring the original calibration parameter values.

In [ ]:
calib_1.gainCalc = calib_1_temp_gC
calib_2.gainCalc = calib_2_temp_gC

rp_hw_calib.cCalibArr_setitem(rp_calib.fast_adc_1_1, 0, calib_1)
rp_hw_calib.cCalibArr_setitem(rp_calib.fast_adc_1_1, 1, calib_2)

rp_hw_calib.rp_CalibrationWriteParams(rp_calib,False)

rp_calib = rp_hw_calib.rp_GetCalibrationSettings()

calib_1 = rp_hw_calib.cCalibArr_getitem(rp_calib.fast_adc_1_1,0)  # Channels 0,1,2,3
calib_2 = rp_hw_calib.cCalibArr_getitem(rp_calib.fast_adc_1_1,1)

print(calib_1.gainCalc)
print(calib_2.gainCalc)

Restoring settings from backup. For the changes to take effect, please restart JupyterLab.

In [ ]:
rp_hw_calib.rp_CalibrationWriteParams(current_settings,False)